# connect


In [21]:
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine, MetaData, URL, Identity
from sqlalchemy import Table, Column, Integer, String, BOOLEAN, DATE, ForeignKey


#Creating Database,Connection
meta = MetaData()
USERNAME = 'root'
PASSWORD = '13771377Mnn@'
SERVER = 'localhost'
PORT = 3306
DATABASE = 'book_store'

url_object = URL.create("mysql+mysqlconnector",
                        username=USERNAME,
                        password=PASSWORD,
                        host=SERVER,
                        database=DATABASE)
engine = create_engine(url_object,echo = True)
conn = engine.connect()

2023-09-26 23:29:33,463 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2023-09-26 23:29:33,465 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-26 23:29:33,470 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2023-09-26 23:29:33,472 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-26 23:29:33,477 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2023-09-26 23:29:33,479 INFO sqlalchemy.engine.Engine [raw sql] {}


# create model database

In [22]:
#Creating Tables
#book
book = Table(
    'book', meta,
    Column('code', Integer, primary_key=True),
    Column('title_persian', String(50)),
    Column('title_english', String(50)),
    Column('price', Integer),
    Column('discount', Integer),
    Column('grade', Integer),
    Column('shabak', String(30)),
    Column('page_count', Integer),
    Column('release_year_sh', Integer),
    Column('release_year_mi', Integer),
    Column('exist', BOOLEAN),
    Column('earliest_access', DATE),
    Column('print_series', Integer),
    Column('cover', String(50)),
    Column('ghate', String(50))
)
meta.create_all(engine)

2023-09-26 23:29:33,577 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-26 23:29:33,579 INFO sqlalchemy.engine.Engine DESCRIBE `book_store`.`book`
2023-09-26 23:29:33,580 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-26 23:29:33,589 INFO sqlalchemy.engine.Engine 
CREATE TABLE book (
	code INTEGER NOT NULL AUTO_INCREMENT, 
	title_persian VARCHAR(50), 
	title_english VARCHAR(50), 
	price INTEGER, 
	discount INTEGER, 
	grade INTEGER, 
	shabak VARCHAR(30), 
	page_count INTEGER, 
	release_year_sh INTEGER, 
	release_year_mi INTEGER, 
	exist BOOL, 
	earliest_access DATE, 
	print_series INTEGER, 
	cover VARCHAR(50), 
	ghate VARCHAR(50), 
	PRIMARY KEY (code)
)


2023-09-26 23:29:33,590 INFO sqlalchemy.engine.Engine [no key 0.00112s] {}
2023-09-26 23:29:33,657 INFO sqlalchemy.engine.Engine COMMIT


In [23]:
#group
group = Table(
    'group', meta,
    Column('id', Integer, primary_key=True),
    Column('description', String(500))
)
meta.create_all(engine)

2023-09-26 23:29:33,702 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-26 23:29:33,703 INFO sqlalchemy.engine.Engine DESCRIBE `book_store`.`book`
2023-09-26 23:29:33,705 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-26 23:29:33,709 INFO sqlalchemy.engine.Engine DESCRIBE `book_store`.`group`
2023-09-26 23:29:33,710 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-26 23:29:33,713 INFO sqlalchemy.engine.Engine 
CREATE TABLE `group` (
	id INTEGER NOT NULL AUTO_INCREMENT, 
	description VARCHAR(500), 
	PRIMARY KEY (id)
)


2023-09-26 23:29:33,715 INFO sqlalchemy.engine.Engine [no key 0.00221s] {}
2023-09-26 23:29:33,746 INFO sqlalchemy.engine.Engine COMMIT


In [24]:
#group_book
group_book = Table(
    'group_book', meta,
    Column('id', Integer, primary_key=True, autoincrement=True),
    Column('group_id', Integer, ForeignKey('group.id'))
)
meta.create_all(engine)

2023-09-26 23:29:33,863 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-26 23:29:33,869 INFO sqlalchemy.engine.Engine DESCRIBE `book_store`.`book`
2023-09-26 23:29:33,872 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-26 23:29:33,885 INFO sqlalchemy.engine.Engine DESCRIBE `book_store`.`group`
2023-09-26 23:29:33,892 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-26 23:29:33,901 INFO sqlalchemy.engine.Engine DESCRIBE `book_store`.`group_book`
2023-09-26 23:29:33,904 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-26 23:29:33,908 INFO sqlalchemy.engine.Engine 
CREATE TABLE group_book (
	id INTEGER NOT NULL AUTO_INCREMENT, 
	group_id INTEGER, 
	PRIMARY KEY (id), 
	FOREIGN KEY(group_id) REFERENCES `group` (id)
)


2023-09-26 23:29:33,909 INFO sqlalchemy.engine.Engine [no key 0.00107s] {}
2023-09-26 23:29:33,969 INFO sqlalchemy.engine.Engine COMMIT


In [26]:
#person
person = Table(
    'person', meta,
    Column('counter', Integer, primary_key=True,autoincrement=True),
    Column('id', Integer),
    Column('name', String(50)),
    Column('description', String(500))
)
meta.create_all(engine)

2023-09-26 23:33:05,378 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-26 23:33:05,381 INFO sqlalchemy.engine.Engine DESCRIBE `book_store`.`book`
2023-09-26 23:33:05,384 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-26 23:33:05,393 INFO sqlalchemy.engine.Engine DESCRIBE `book_store`.`group`
2023-09-26 23:33:05,394 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-26 23:33:05,397 INFO sqlalchemy.engine.Engine DESCRIBE `book_store`.`group_book`
2023-09-26 23:33:05,398 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-26 23:33:05,402 INFO sqlalchemy.engine.Engine DESCRIBE `book_store`.`person`
2023-09-26 23:33:05,403 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-26 23:33:05,406 INFO sqlalchemy.engine.Engine 
CREATE TABLE person (
	counter INTEGER NOT NULL AUTO_INCREMENT, 
	id INTEGER, 
	name VARCHAR(50), 
	description VARCHAR(500), 
	PRIMARY KEY (counter)
)


2023-09-26 23:33:05,407 INFO sqlalchemy.engine.Engine [no key 0.00125s] {}
2023-09-26 23:33:05,425 INFO sqlalch

In [27]:
#publisher
publisher = Table(
    'publisher', meta,
    Column('id', Integer, primary_key=True),
    Column('name', String(50))
)
meta.create_all(engine)

2023-09-26 23:33:07,547 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-26 23:33:07,549 INFO sqlalchemy.engine.Engine DESCRIBE `book_store`.`book`
2023-09-26 23:33:07,550 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-26 23:33:07,557 INFO sqlalchemy.engine.Engine DESCRIBE `book_store`.`group`
2023-09-26 23:33:07,558 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-26 23:33:07,563 INFO sqlalchemy.engine.Engine DESCRIBE `book_store`.`group_book`
2023-09-26 23:33:07,564 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-26 23:33:07,567 INFO sqlalchemy.engine.Engine DESCRIBE `book_store`.`person`
2023-09-26 23:33:07,568 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-26 23:33:07,575 INFO sqlalchemy.engine.Engine DESCRIBE `book_store`.`publisher`
2023-09-26 23:33:07,576 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-26 23:33:07,581 INFO sqlalchemy.engine.Engine 
CREATE TABLE publisher (
	id INTEGER NOT NULL AUTO_INCREMENT, 
	name VARCHAR(50), 
	PRIMARY KEY (id)
)


2023-

In [28]:
#category
category = Table(
    'category', meta,
    Column('id', Integer, primary_key=True),
    Column('name', String(50)),
    Column('description', String(500))
)
meta.create_all(engine)

2023-09-26 23:33:09,720 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-26 23:33:09,726 INFO sqlalchemy.engine.Engine DESCRIBE `book_store`.`book`
2023-09-26 23:33:09,729 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-26 23:33:09,734 INFO sqlalchemy.engine.Engine DESCRIBE `book_store`.`group`
2023-09-26 23:33:09,734 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-26 23:33:09,742 INFO sqlalchemy.engine.Engine DESCRIBE `book_store`.`group_book`
2023-09-26 23:33:09,743 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-26 23:33:09,749 INFO sqlalchemy.engine.Engine DESCRIBE `book_store`.`person`
2023-09-26 23:33:09,750 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-26 23:33:09,756 INFO sqlalchemy.engine.Engine DESCRIBE `book_store`.`publisher`
2023-09-26 23:33:09,758 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-26 23:33:09,763 INFO sqlalchemy.engine.Engine DESCRIBE `book_store`.`category`
2023-09-26 23:33:09,764 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-26 

In [29]:
#group_category
group_category = Table(
    'group_category', meta,
    Column('id', Integer, primary_key=True, autoincrement=True),
    Column('group_id', Integer, ForeignKey('group.id')),
    Column('category_id', Integer, ForeignKey('category.id'))
)
meta.create_all(engine)

2023-09-26 23:33:12,219 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-26 23:33:12,223 INFO sqlalchemy.engine.Engine DESCRIBE `book_store`.`book`
2023-09-26 23:33:12,225 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-26 23:33:12,234 INFO sqlalchemy.engine.Engine DESCRIBE `book_store`.`group`
2023-09-26 23:33:12,235 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-26 23:33:12,240 INFO sqlalchemy.engine.Engine DESCRIBE `book_store`.`group_book`
2023-09-26 23:33:12,241 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-26 23:33:12,244 INFO sqlalchemy.engine.Engine DESCRIBE `book_store`.`person`
2023-09-26 23:33:12,245 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-26 23:33:12,248 INFO sqlalchemy.engine.Engine DESCRIBE `book_store`.`publisher`
2023-09-26 23:33:12,249 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-26 23:33:12,252 INFO sqlalchemy.engine.Engine DESCRIBE `book_store`.`category`
2023-09-26 23:33:12,253 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-26 

In [30]:
#book_publisher
book_publisher = Table(
    'book_publisher', meta,
    Column('id', Integer, primary_key=True, autoincrement=True),
    Column('book_code', Integer, ForeignKey('book.code')),
    Column('publisher_id', Integer, ForeignKey('publisher.id'))
)
meta.create_all(engine)

2023-09-26 23:33:15,984 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-26 23:33:15,986 INFO sqlalchemy.engine.Engine DESCRIBE `book_store`.`book`
2023-09-26 23:33:15,987 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-26 23:33:15,993 INFO sqlalchemy.engine.Engine DESCRIBE `book_store`.`group`
2023-09-26 23:33:15,995 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-26 23:33:15,999 INFO sqlalchemy.engine.Engine DESCRIBE `book_store`.`group_book`
2023-09-26 23:33:16,000 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-26 23:33:16,003 INFO sqlalchemy.engine.Engine DESCRIBE `book_store`.`person`
2023-09-26 23:33:16,004 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-26 23:33:16,010 INFO sqlalchemy.engine.Engine DESCRIBE `book_store`.`publisher`
2023-09-26 23:33:16,011 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-26 23:33:16,014 INFO sqlalchemy.engine.Engine DESCRIBE `book_store`.`category`
2023-09-26 23:33:16,015 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-26 

In [31]:
#crew
crew = Table(
    'crew', meta,
    Column('id', Integer, primary_key=True, autoincrement=True),
    Column('book_code', Integer, ForeignKey('book.code')),
    Column('person_counter', Integer, ForeignKey('person.counter')),
    Column('role', String(15))
)
meta.create_all(engine)

2023-09-26 23:33:19,434 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-26 23:33:19,439 INFO sqlalchemy.engine.Engine DESCRIBE `book_store`.`book`
2023-09-26 23:33:19,443 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-26 23:33:19,454 INFO sqlalchemy.engine.Engine DESCRIBE `book_store`.`group`
2023-09-26 23:33:19,456 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-26 23:33:19,461 INFO sqlalchemy.engine.Engine DESCRIBE `book_store`.`group_book`
2023-09-26 23:33:19,462 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-26 23:33:19,465 INFO sqlalchemy.engine.Engine DESCRIBE `book_store`.`person`
2023-09-26 23:33:19,466 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-26 23:33:19,471 INFO sqlalchemy.engine.Engine DESCRIBE `book_store`.`publisher`
2023-09-26 23:33:19,472 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-26 23:33:19,480 INFO sqlalchemy.engine.Engine DESCRIBE `book_store`.`category`
2023-09-26 23:33:19,483 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-09-26 